<a href="https://colab.research.google.com/github/dgsmith1988/ECSE-552-Final-Project/blob/rubert/execution_framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overall Script Description
Run through each of the different cells to get things working. What should happen is that you connect to Google Drive, some data is transferred and then unzipped and then the model is trained on it and outputs data indicating correct operation. On the Google Drive there should also be files added under "Measurements" which have a date and time-stamp corresponding to the output of training process. The time stamps are generated by the Google Colab instance so they won't match exactly with your local time depending on where the instance they supplied us is located.

The cells are broken up based on functionality to make it easier to debug/profile/troubleshoot any issues which might come up.

# Configure Directories/Paths/Languages
The following cell is where you configure which languages you want to work with (from the three which are available on Google Drive at the moment). It is also where you set the paths to where the Google Drive will be mounted and where the data is extracted to.


In [3]:
import os
languages = ["EN", "DE", "ES"]
mount_point = '/content/drive'
load_path = os.path.join(mount_point, 'MyDrive/ECSE-552-FP/Data/')
unzip_path = '/content/speech_data'
save_path = '/MyDrive/ECSE-552-FP/Measurements'

from google.colab import drive
drive.mount(mount_point,force_remount= True)

Mounted at /content/drive


# Download and extract the dataset zip files
The following downloads and extracts the language zip files from Google Drive. It is done separately in it's own cell to make it easier to profile/debug things if there are issues in the network connection between Google Drive and Google Colab. 

This is separate from dataset creation for two reasons. The first is that if the network accesses are obscured by accessing the data via the network drive, it is much harder to debug bottlenecks in creating the dataset. The second is that all of the data must be transferred once at somepoint during one epoch, so it's easier to do it all up front and make sure the data is on the Colab instance before training. Localizing as much as possible to the Colab instance during training makes things easier to debug as well as more efficient.

There is a commented out code which allows you to select between the "debug" dataset and the "full" dataset for the languages. The "debug" set consists of a smaller subset (20 samples for each language) to aid in debugging the dataflow in models. It alleviates the need to run through the entire dataset and helps ensure all the pipes are connected correctly.

In [4]:
for language in languages:
    language_dir_path = os.path.join(unzip_path, language)
    os.makedirs(language_dir_path, exist_ok=True)
    archive = language + "_debug_set" + ".zip"
    #archive = language + ".zip"
    !unzip {os.path.join(load_path, archive)} -d {language_dir_path}

Archive:  /content/drive/MyDrive/ECSE-552-FP/Data/EN_debug_set.zip
   creating: /content/speech_data/EN/1snoke-20120412-hge/
  inflating: /content/speech_data/EN/1snoke-20120412-hge/a0405.wav  
  inflating: /content/speech_data/EN/1snoke-20120412-hge/a0406.wav  
  inflating: /content/speech_data/EN/1snoke-20120412-hge/a0407.wav  
  inflating: /content/speech_data/EN/1snoke-20120412-hge/a0408.wav  
  inflating: /content/speech_data/EN/1snoke-20120412-hge/a0409.wav  
  inflating: /content/speech_data/EN/1snoke-20120412-hge/a0410.wav  
  inflating: /content/speech_data/EN/1snoke-20120412-hge/a0411.wav  
  inflating: /content/speech_data/EN/1snoke-20120412-hge/a0412.wav  
  inflating: /content/speech_data/EN/1snoke-20120412-hge/a0413.wav  
  inflating: /content/speech_data/EN/1snoke-20120412-hge/a0414.wav  
   creating: /content/speech_data/EN/Bahoke-20130721-uop/
  inflating: /content/speech_data/EN/Bahoke-20130721-uop/b0520.wav  
  inflating: /content/speech_data/EN/Bahoke-20130721-uop/b

# Download the source code from GitHub

In [1]:
user = "dgsmith1988"
repo = "ECSE-552-Final-Project"
src_dir = "Code"
pyfiles = ["models.py", "train.py", "dict_logger.py", "feature_extraction.py"]

for pyfile in pyfiles:
    !rm {pyfile}
    url = f"https://raw.githubusercontent.com/{user}/{repo}/rubert/{src_dir}/{pyfile}"
    !wget {url}


rm: cannot remove 'models.py': No such file or directory
--2022-03-21 20:54:34--  https://raw.githubusercontent.com/dgsmith1988/ECSE-552-Final-Project/rubert/Code/models.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5628 (5.5K) [text/plain]
Saving to: ‘models.py’

models.py           100%[===================>]   5.50K  --.-KB/s    in 0s      

2022-03-21 20:54:34 (35.9 MB/s) - ‘models.py’ saved [5628/5628]

rm: cannot remove 'train.py': No such file or directory
--2022-03-21 20:54:34--  https://raw.githubusercontent.com/dgsmith1988/ECSE-552-Final-Project/rubert/Code/train.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercont

# Install the supporting/required libraries



In [5]:
!pip install torchmetrics
!pip install pytorch-lightning
!pip install librosa

     |████████████████████████████████| 397 kB 5.3 MB/s 
     |████████████████████████████████| 527 kB 5.2 MB/s 
     |████████████████████████████████| 829 kB 48.8 MB/s 
     |████████████████████████████████| 952 kB 40.7 MB/s 
     |████████████████████████████████| 134 kB 33.5 MB/s 
     |████████████████████████████████| 596 kB 42.4 MB/s 
     |████████████████████████████████| 1.1 MB 41.4 MB/s 
     |████████████████████████████████| 271 kB 36.9 MB/s 
     |████████████████████████████████| 144 kB 47.1 MB/s 
     |████████████████████████████████| 94 kB 2.8 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=51e3493783eee4f5ebbdb88ebbb7326ac141d1f2db7f82eb677e715a21e59cd9
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfull

# Hyperparameter tuning

In [6]:
!pip install optuna

     |████████████████████████████████| 308 kB 5.3 MB/s 
     |████████████████████████████████| 210 kB 36.3 MB/s 
     |████████████████████████████████| 81 kB 7.7 MB/s 
     |████████████████████████████████| 78 kB 3.8 MB/s 
     |████████████████████████████████| 150 kB 34.5 MB/s 
     |████████████████████████████████| 49 kB 2.9 MB/s 
     |████████████████████████████████| 113 kB 32.8 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=1ad1da5e8409a52bd25a8e94ca391b9e06902a7572e6068d1b281583b7d6517c
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [7]:
import optuna
import train
from models import BaselineResnetClassifier

model = BaselineResnetClassifier(num_classes=3)
train.hp_tuning_voxforge_classifier(model, data_dir=unzip_path)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

[I 2022-03-21 20:56:20,311] A new study created in memory with name: no-name-02972079-f4f7-4f3e-8a23-5fb22bc960ad


Preparing and splitting dataset...


3it [00:00,  6.21it/s]
3it [00:00,  5.24it/s]
3it [00:01,  2.89it/s]
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:148: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=False)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=False)`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

[I 2022-03-21 20:57:20,638] Trial 9 finished with value: 1.0 and parameters: {'max_t': 4, 'batch_size': 17}. Best is trial 2 with value: 1.0.


Number of finished trials: 10
Best trial:
  Value: 1.0
  Params: 
    max_t: 5
    batch_size: 16


# Run/Train the Model

In [ ]:
import train
from models import BaselineResnetClassifier

model = BaselineResnetClassifier(num_classes=3)
train.train_voxforge_classifier(model, data_dir=unzip_path)

# Save model to Google Drive

In [ ]:
# When a model needs to be saved for future analysis, it must be moved to Google drive, since colab is not-persistent

In [ ]:
# First, download the move_checkpoint.py script
user = "dgsmith1988"
repo = "ECSE-552-Final-Project"
src_dir = "Code"
pyfiles = ["move_checkpoint.py"]

for pyfile in pyfiles:
    !rm {pyfile}
    url = f"https://raw.githubusercontent.com/{user}/{repo}/rubert/{src_dir}/{pyfile}"
    !wget {url}

rm: cannot remove 'move_checkpoint.py': No such file or directory
--2022-03-19 16:44:25--  https://raw.githubusercontent.com/dgsmith1988/ECSE-552-Final-Project/rubert/Code/move_checkpoint.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1420 (1.4K) [text/plain]
Saving to: ‘move_checkpoint.py’

move_checkpoint.py  100%[===================>]   1.39K  --.-KB/s    in 0s      

2022-03-19 16:44:25 (23.8 MB/s) - ‘move_checkpoint.py’ saved [1420/1420]



In [ ]:
# NExt, move the desired file, and assigne a new meanigful_name. 
# it will be properly stored in the folder Checkpoints
from move_checkpoint import move_checkpoint
move_checkpoint(r'/content/DictLogger/0.1/checkpoints/epoch=4-step=10989.ckpt', 'first_step')

Running on Colab


In [ ]:
# TODO: which hyperparameters parmeters should be stored?
# TODO: do the checkpoints contain the required hyperparameters of a particular model? If not, is there a way to save these hyperparameters?
# TODO: what are the best ways to find the optimal parameters? Cross validation?
# TODO: should I include other metrics beyond loss?